# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
ratings = pd.pivot_table(df, index='ProductName',  values='Quantity', columns='CustomerID', aggfunc='sum', fill_value=0)
ratings.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [6]:
ratingsT = ratings.transpose()
ratingsT.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [7]:
ratingsT_dist = ratingsT.div(ratingsT.sum(axis=1), axis=0)
ratingsT_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.015873,0.000000,0.000000,0.0,0.0,0.015873,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.015873,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.012987,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012987,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.015385,0.015385,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.015385,0.0,0.0,0.0,0.000000,0.000000,0.0
356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.014925,0.0
412,0.0,0.0,0.0,0.0,0.013699,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.013699,0.013699,0.013699,0.0,0.0,0.0,0.000000,0.000000,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
dist_eucl = 1/(1+squareform(pdist(ratingsT_dist, metric = 'euclidean')))
cust_sim_matrix = pd.DataFrame(dist_eucl, columns=ratingsT_dist.index, index=ratingsT_dist.index)
cust_sim_matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [9]:
# iterate for first customerID (33) 
top5_similar_customers = cust_sim_matrix[33].sort_values(ascending=False)[1:6]
print("Similarities for customer 33: \n")
for custID, value in dict(top5_similar_customers).items():
    print(f'Customer {custID} value {value}')


Similarities for customer 33: 

Customer 60862 value 0.8705439997643089
Customer 27672 value 0.8670105783072379
Customer 6001 value 0.8646724750120224
Customer 79458 value 0.8646711828039201
Customer 33759 value 0.8643828209626243


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [10]:
similar_customers_products = df.loc[top5_similar_customers.index]
similar_customers_products.head()

C:\Users\lgutierrez\AppData\Local\r-miniconda\envs\r-reticulate\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,CustomerID,ProductName,Quantity
CustomerID,,,
60862,93882.0,Watercress,24.0
27672,42819.0,"Turnip - White, Organic",11.0
6001,8539.0,Olives - Stuffed,3.0
79458,NaN,NaN,NaN
33759,52479.0,"Beans - Kidney, Canned",14.0


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [11]:
ranked = similar_customers_products.groupby('ProductName')['Quantity'].agg({'Quantity':'sum'}).sort_values(by=['Quantity'], ascending=False)
ranked = ranked.reset_index()
ranked.head()

C:\Users\lgutierrez\AppData\Local\r-miniconda\envs\r-reticulate\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,ProductName,Quantity
0,Watercress,24.0
1,"Beans - Kidney, Canned",14.0
2,"Turnip - White, Organic",11.0
3,Olives - Stuffed,3.0


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [12]:
ratings_fill =ratings[[33]].reset_index()
merged = pd.merge(ranked, ratings_fill, on='ProductName')
merged.columns = ['ProductName','Quantity', 'Purchased']
recommended = merged[merged['Purchased']==0][['Quantity']]
recommended.head()

,Quantity
0,24.0
2,11.0
3,3.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
recomendations = {}
customers = df.CustomerID.unique()
customers = customers[:100]
for customer in customers:
    similar_cust = cust_sim_matrix[customer].sort_values(ascending=False)[0:5]
    similar_cust_product = df.loc[similar_cust.index]
    
    ranked = similar_cust_product.groupby('ProductName').agg({'Quantity':'sum'}).sort_values(by=['Quantity'], ascending=False)
    ranked = ranked.reset_index()

  
    ratings_fill =ratings[customer].reset_index()
    merged = pd.merge(ranked, ratings_fill, on='ProductName')
    merged.columns = ['ProductName','Quantity', 'Purchased']
  
    recomendations = merged[merged['Purchased']==0][['Quantity']]
    recomendations[customer] = list(recomendations.index[0:5])
recomendations

C:\Users\lgutierrez\AppData\Local\r-miniconda\envs\r-reticulate\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


,Quantity,9260
0,23.0,0
1,23.0,1
2,19.0,2
3,4.0,3


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [17]:
recomendations
# recomendations2 = recomendations.reset_index()
# recomendations2.columns = ['CustomerID', 'Rec1','Rec2','Rec3','Rec4','Rec5']
# recomendations2 = recomendations2.set_index('CustomerID')
# recomendations2.head()

ValueError: Length mismatch: Expected axis has 3 elements, new values have 6 elements

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.